# Extract data from MariaDB and save it in ndjson files

### Import librairies

In [66]:
import mariadb # librairie pour se connecter à mariadb
import sys
from extraction import var # import variable configured into var.py
import pandas as pd
import random
import os
import json

### Connect to database

In [1]:

# Connect to MariaDB Platform
try:
    conn = mariadb.connect(
        user=var.user,
        password=var.password,
        host=var.host,
        port=3307,
    )
except mariadb.Error as e:
    print(f"Error connecting to MariaDB Platform: {e}")
    sys.exit(1)

# Get Cursor
cur = conn.cursor()


### Retrieve databases on the server
❗❗<code>SHOW DATABASES</code> query requires specific right on mariadb server
If our user do not get those rights, please run the command with another user
Some databases are not supposed to be transfered into Elastic, for exemple : <code>{'information_schema','mysql','sys','performance_schema'}</code>


In [2]:
cur.execute("SHOW DATABASES;") #* require specific rights on mariadb server

result = cur.fetchall() # retrieve all the results

cur.close()
# Create a list of databases name on the server
dbs_list= []
for row in result:
    dbs_list.append(row[0]) 

#! TODO : complete unwanted_db with any database that should not be transfered into Elastic
unwanted_db = {'information_schema','mysql','sys','performance_schema'}

#* dbs_list contain all databases that will be taken into account for batch
dbs_list = [db for db in dbs_list if db not in unwanted_db]

In [3]:
display(dbs_list)

['meteo1', 'mouleconnected']

### Get tables inside Databases
We create a dictionnary <code>dict_db</code> with database name as key and value being a list  of tables inside the database

In [4]:
# for each db we want to transfer, we store tables names in the dictionnary
dict_db = {}
for db in dbs_list:
    cur = conn.cursor()
    query = f"SHOW TABLES FROM {db}"
    cur.execute(query)
    result = cur.fetchall()
    result = [value[0] for value in result] # keep only the name of the table
    dict_db[db] = result
    cur.close()


In [5]:
display(dict_db)

{'meteo1': ['DATA',
  'LOGBOOK',
  'ane0',
  'bar0',
  'cpr0',
  'gps0',
  'gsm0',
  'ntp0',
  'rad0',
  'the0'],
 'mouleconnected': ['Temperature',
  'bac',
  'capteur',
  'data',
  'data2',
  'lumiere',
  'moule',
  'users']}

Convert dict of list into dict of dict with list for values extracted

In [6]:
dict_db_tables = {}
for key, values in dict_db.items():
    dict_db_tables[key] = {value: {} for value in values}
display(dict_db_tables)

{'meteo1': {'DATA': {},
  'LOGBOOK': {},
  'ane0': {},
  'bar0': {},
  'cpr0': {},
  'gps0': {},
  'gsm0': {},
  'ntp0': {},
  'rad0': {},
  'the0': {}},
 'mouleconnected': {'Temperature': {},
  'bac': {},
  'capteur': {},
  'data': {},
  'data2': {},
  'lumiere': {},
  'moule': {},
  'users': {}}}

### Retrieve columns in tables

In [7]:
from copy import deepcopy
dict_col_in_table = deepcopy(dict_db_tables)
for db in dict_db.keys():
    for table in dict_db[db]:
        cur = conn.cursor()
        query = f"SHOW COLUMNS FROM {db}.{table}"
        cur.execute(query)
        result = cur.fetchall()
        cur.close()
        cols = [value[0] for value in result]
        i=0
        for i in range(len(cols)):
            dict_col_in_table[db][table][cols[i]] = []


In [8]:
dict_col_in_table

{'meteo1': {'DATA': {'utctimestamp': [], 'download': []},
  'LOGBOOK': {'utctimestamp': [], 'event': []},
  'ane0': {'utctimestamp': [], 'windspeed': [], 'winddirection': []},
  'bar0': {'utctimestamp': [], 'atmosphericpressure': []},
  'cpr0': {'utctimestamp': [],
   'uptime': [],
   'diskspace': [],
   'ramusage': [],
   'runningprocesses': [],
   'cpuload': []},
  'gps0': {'utctimestamp': [],
   'latitude': [],
   'longitude': [],
   'satellites': [],
   'hdop': [],
   'altitude': []},
  'gsm0': {'utctimestamp': [], 'atstatus': [], 'smsstatus': []},
  'ntp0': {'utctimestamp': [], 'offset': []},
  'rad0': {'utctimestamp': [], 'parirradiance': []},
  'the0': {'utctimestamp': [], 'airtemperature': []}},
 'mouleconnected': {'Temperature': {'IdBac': [], 'IdTemperature': []},
  'bac': {'Description': [], 'IdBac': []},
  'capteur': {'UniteMesure': [],
   'IdMoule': [],
   'IdTemperature': [],
   'IdLumiere': [],
   'IdBac': [],
   'IdCapteur': []},
  'data': {'DateMesure': [],
   'IdCapteu

In [9]:
dict_db_tables

{'meteo1': {'DATA': {},
  'LOGBOOK': {},
  'ane0': {},
  'bar0': {},
  'cpr0': {},
  'gps0': {},
  'gsm0': {},
  'ntp0': {},
  'rad0': {},
  'the0': {}},
 'mouleconnected': {'Temperature': {},
  'bac': {},
  'capteur': {},
  'data': {},
  'data2': {},
  'lumiere': {},
  'moule': {},
  'users': {}}}

In [10]:

#? get columns inside each tables and store it inside a dictionnary with arrays as values
# for db in dict_db.keys():
#     for table in dict_db[db]:
#         cur = conn.cursor()
#         query = f"SHOW COLUMNS FROM {db}.{table}"
#         cur.execute(query)
#         result = cur.fetchall()
#         cols = [value[0] for value in result]
#         i=0
#         for i in range(len(cols)):
#             dict_db_tables[db][table][cols[i]] = []

# display(dict_db_tables)
        

### Extract data from table and insert it in the right column

In [11]:
# for db in dict_db.keys(): # iteration selon les bdd
#     for table in dict_db[db]: # pour chaque tables de chaque BDD
#         cur = conn.cursor()
#         query = f"SELECT * FROM {db}.{table}" # extraction de l'intégralité des données
#         cur.execute(query)
#         result = cur.fetchall()
#         rowNb = 0
#         columnNb = 0
#         for col in dict_db_tables[db][table]: # pour chaque colonne dans la table
#             for rowNb in range(len(result)): # pour chaque ligne dnas la table
#                 dict_db_tables[db][table][col].append(result[rowNb][columnNb]) # on ajoute à la liste la donnée de la ligne correspondant à la colonne actuelle
#             columnNb+=1 # on passe à la colonne suivante
            


We get a dictionnary containing all the databases ; with their tables ; column inside tables and finaly all values per columns like this : 
<pre>
<code>
{
    database1 : {
        table1 : {
            column1 : [value1,value2, ..., valueN],
            column2 : [value1,value2, ..., valueM],
            ...
        },
        table2 : {
            column1 : [value1,value2, ..., valueN],
            column2 : [value1,value2, ..., valueM],
            ...
        },
        ...
    }
}
</code>
</pre>


<b>For databases that contain timestamp with value in each tables like meteo1 we can directly insert data from each tables into Elastic because the data will be studied 'separetely'.
For database with relation between tables like mouleconnected we have to join tables</b>

### Define list of databases that requires to join tables and which that doesn't need to there tables to be joined

In [12]:
list_db_nojoin = ['meteo1']
list_db_withjoin = [ x for x in dbs_list if x not in list_db_nojoin] # from the list containing all databases we retrieve only db that are not those that do not require join

### Extraction for database with disjoined tables with utctimestamp

In [13]:

#! TODO create two lists of databases, one for db with disjoined tables ; one db with tables to join 
for db in list_db_nojoin:
    for table in dict_db[db]: # pour chaque tables de chaque BDD
        cur = conn.cursor()
        query = f"SELECT * FROM {db}.{table} ORDER BY utctimestamp ASC " # extraction de l'intégralité des données
        cur.execute(query)
        result = cur.fetchall()
        cur.close()
        rowNb = 0
        for rowNb in range(len(result)): # pour chaque ligne dans la table
            item = f'item_{rowNb}'
            dict_db_tables[db][table][item] = {}
            columnNb = 0
            for col in dict_col_in_table[db][table]: # pour chaque colonne dans la table
                dict_db_tables[db][table][item][col] = result[rowNb][columnNb] # on ajoute à la liste la donnée de la ligne correspondant à la colonne actuelle
                columnNb+=1 # on passe à la colonne suivante

In [14]:
dict_db_tables['meteo1']['ane0']['item_14713880']

{'utctimestamp': 20121218081745, 'windspeed': 11.3, 'winddirection': 264.0}

### Save data from databases that do not require join in ndjson format

#### Create a directory per db and a file per table 

In [82]:
# select a batch_size
def select_batch_size(n):
    diviseurs = []
    for i in range(20, 500000):
        if n % i == 0:
            diviseurs.append(i)
            print(i)
    return random.choice(diviseurs)

In [83]:
select_batch_size(194998)

97499
194998


97499

In [ ]:
# fonction qui convertit les nan en None car les nan ne sont pas supportés en json
def nan_serializer(obj):
        if pd.isna(obj):
                return None
        return obj

In [87]:

current_dir = os.getcwd()
for db in list_db_nojoin:
    folder_path = current_dir + '/' + db
    os.makedirs(folder_path, exist_ok=True)
    for table in dict_db_tables[db] :
        file_path = f'{db}/{table}.ndjson'
        lenght = len(dict_db_tables[db][table].items())
        if lenght <= 500000:
            for key, value in dict_db_tables[db][table].items():
                with open(file_path, 'a') as f:
                    f.write(json.dumps({key:value}) + '\n')
        else :
            batch_size = select_batch_size(lenght)
            # on crée une liste contenant chacun des éléments du dictionnaire afin de pouvoir les parcourir 
            items = list(dict_db_tables[db][table].items())
            batches = [items[i:i+batch_size] for i in range(0,lenght, batch_size)]
            # on parcourt les lots
            for i, batch in enumerate(batches):
                # on enregistre le contenu du lot dans un fichier
                with open(file_path, 'a') as f:
                    for key, value in batch:
                        entry = {key:value}
                        f.write(json.dumps(entry, default=nan_serializer) + '\n')

### Joining tables for databases with relation between tables beyond timestamp

#### Extract the data from all useful tables in mouleconnected database, store it into dataframe stored in a dictionnary

In [15]:
df_list = {}
for db in list_db_withjoin:
    df_list[db]={}
    for table in dict_db[db]:
        if db == 'mouleconnected':
            if table in ['data', 'data2', 'capteur', 'bac']:
                cur = conn.cursor()
                query = f"SELECT * FROM {db}.{table}"
                cur.execute(query)
                result = cur.fetchall()
                columns = [col for col in dict_col_in_table[db][table].keys()]
                cur.close()
                if (table == 'data' or table =='data2'):
                    df = pd.DataFrame(result, columns=columns, dtype=pd.Int64Dtype())
                elif table == "capteur":
                    df = pd.DataFrame(result, columns=columns).astype({'UniteMesure':str, 'IdMoule': pd.Int64Dtype(), 'IdTemperature': pd.Int64Dtype(), 'IdLumiere': pd.Int64Dtype(), 'IdCapteur':pd.Int64Dtype(), 'IdBac': pd.Int64Dtype()})
                else:
                    df = pd.DataFrame(result, columns=columns).astype({'IdBac': int, 'Description': str}) 
                df_list[db][table] = df

In [16]:
df_list['mouleconnected']['data']

,DateMesure,IdCapteur,ValMesure,ConvTimeStamp,IdData
0,20200825141815,1,2557,1598365095000,1
1,20200825141815,2,2833,1598365095000,2
2,20200825141815,3,3032,1598365095000,3
3,20200825141815,4,2765,1598365095000,4
4,20200825141815,5,2361,1598365095000,5
...,...,...,...,...,...
24519043,20200907090210,104,0,1599469329000,24519044
24519044,20200907090210,201,1750,1599469329000,24519045
24519045,20200907090210,202,1725,1599469329000,24519046
24519046,20200907090210,203,-6,1599469329000,24519047


#### Join tables of mouleconnected db into a unique dataframe

In [17]:
def join_mouleconnected():
    # drop unused column from tables
    if 'IdData' in df_list['mouleconnected']['data']:
        data = df_list['mouleconnected']['data'].drop(['IdData', 'DateMesure'], axis=1)
        data2 = df_list['mouleconnected']['data2'].drop(['IdData', 'DateMesure'], axis=1)
    else :
        data = df_list['mouleconnected']['data']
        data2 = df_list['mouleconnected']['data2']

    capteur = df_list['mouleconnected']['capteur']
    bac = df_list['mouleconnected']['bac']
    # concatenate the content of data and data2
    data_df = pd.concat([data, data2], ignore_index=True)

    # join data_df with capteur
    data_df = pd.merge(data_df, capteur, on='IdCapteur', how='right')
    # join data_df with bac
    data_df = pd.merge(data_df, bac, on='IdBac', how='right')
    return data_df

#### Convert the dataframe into json for easy insertion in ElasticSearch

In [18]:
test = join_mouleconnected()

In [21]:
test.sort_values(by='ConvTimeStamp', ascending=False, inplace=True)
display(test.head())
display(test.info())

,IdCapteur,ValMesure,ConvTimeStamp,UniteMesure,IdMoule,IdTemperature,IdLumiere,IdBac,Description
2044066,2,2284,1618423247000,Volt,2,<NA>,<NA>,1,test
17375116,103,178,1618423247000,Volt,<NA>,<NA>,103,3,Test
22485466,16,2276,1618423247000,Volt,16,<NA>,<NA>,4,
1021996,1,2282,1618423247000,Volt,1,<NA>,<NA>,1,test
11242696,102,1713,1618423247000,Volt,<NA>,<NA>,102,2,<br />kjihih


<class 'pandas.core.frame.DataFrame'>
Index: 24529682 entries, 2044066 to 24529681
Data columns (total 9 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   IdCapteur      Int64 
 1   ValMesure      Int64 
 2   ConvTimeStamp  Int64 
 3   UniteMesure    object
 4   IdMoule        Int64 
 5   IdTemperature  Int64 
 6   IdLumiere      Int64 
 7   IdBac          Int64 
 8   Description    object
dtypes: Int64(7), object(2)
memory usage: 2.0+ GB


None

#### Works but very long to compute (From 15 to 30minutes)

In [22]:
mouleconnected = {}
i = 0
mouleconnected['state'] = 0
for index, row in test.iterrows():
    if pd.notna(row['ConvTimeStamp']):
        if row['ConvTimeStamp'] != mouleconnected['state']:
            mouleconnected['state'] = row['ConvTimeStamp']
            item = f'item_{i}'
            i+=1
            mouleconnected[item] = {}
            mouleconnected[item]['timestamp'] = row['ConvTimeStamp']
            mouleconnected[item]['records'] = {}
            record_nb = 0
        else:
            record_nb += 1
        record_value = f'record_{record_nb}'
        mouleconnected[item]['records'][record_value]={}
        for col, value in row.items():
            if col != 'ConvTimeStamp':
                mouleconnected[item]['records'][record_value][col] = value 


In [23]:
len(mouleconnected)

1022014

#### Save to json file

In [55]:

# on crée une liste contenant chacun des éléments du dictionnaire afin de pouvoir les parcourir 
items = list(mouleconnected.items())

# on crée des lots de 74 éléments du dictionnaire
batches = [items[i:i+74] for i in range(1,len(mouleconnected), 74)]

# on parcourt les lots
for i, batch in enumerate(batches):
    # on enregistre le contenu du lot dans un fichier
    with open('mouleconnected.ndjson', 'a') as f:
        for key, value in batch:
            entry = {key:value}
            f.write(json.dumps(entry, default=nan_serializer) + '\n')

#### Alternative en cours pour réalister la même chose mais en plusieurs étapes

In [ ]:
# mouleconnectedBatch = {}
# i = 0
# mouleconnected['state'] = 0
# for group_key, group_chunk in test.groupby(by='ConvTimeStamp'):
#     print(group_key, group_chunk)
